In [20]:

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
#设置训练的超参数
lr = 0.001
training_iters = 100000
batch_size = 128

#神经网络的参数
n_inputs = 28 #输出层的n
n_steps = 28 #28长度
n_hidden_units = 128 #隐藏层的神经元个数
n_classes = 10 #输出数量，即分类的类别，0～9个数字，共有10个

#定义输入数据及权重
#输出数据占位符
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

#定义权重
weights = {
    #(28,128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    #(128, 10)
    'out':tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}

biases = {
    #(128, )
    'in': tf.Variable(tf.constant(0.1, shape = [n_hidden_units, ])),
    #(10, )
    'out':tf.Variable(tf.constant(0.1, shape = [n_classes, ]))
}

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [21]:
#定义RNN模型：
def RNN(X, weights, biases):
    #把输入的X转换成X==> (128 batch*28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])
    
    #进入隐藏层
    #X_in = (128 batch * 28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']
    #X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    
    # 这里采用基本的LSTM循环网络单元： Basic LSTM Cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias = 1.0,
                                             state_is_tuple = True)
    #初始化为零值, lstm单元由两个部分组成: (c_state, h_state)
    init_state = lstm_cell.zero_state(batch_size, dtype = tf.float32)
    
    #dynamic_rnn 接受张量(batch, steps, inputs)或者(steps, batch, inputs)作为X_in
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state = init_state, time_major = False)
    results = tf.matmul(final_state[1], weights['out'] + biases['out'])
    return results

In [25]:
#定义损失函数和优化器,优化器采用 AdamOptimizer
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

#定义模型预测结果及准确率计算方式:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#训练数据及评估模型
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict = {
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict = {
                x:batch_xs,
                y:batch_ys,
            }))
            step += 1

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-6-a707712dd0cc>", line 20, in RNN
    init_state, time_major = False)
  File "<ipython-input-7-d535be82c388>", line 2, in <module>
    pred = RNN(x, weights, biases)
  File "/home/wangzehong/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
#训练数据及评估模型
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict = {
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict = {
                x:batch_xs,
                y:batch_ys,
            }))
            step += 1

NameError: name 'train_op' is not defined